In [4]:
# loading in modules
import sqlite3
import sys
# from peewee import *
import pandas as pd 
from sudulunu.helpers import pp, make_num, dumper, rc

from transformers import pipeline

## Zero shot classifer
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")
# https://huggingface.co/facebook/bart-large-mnli

## Text classification/clustering
# https://huggingface.co/docs/transformers/tasks/sequence_classification

import os 
import pathlib
pathos = pathlib.Path(os.path.abspath('zero_shot.ipynb')).parent
os.chdir(pathos)
print(os.getcwd())


/Users/josh_nicholas/Repos/oz-2023-facebook-political-ads-scraper/classify_model


In [5]:

db = 'input/scraperwiki.sqlite'
con = sqlite3.connect(db)

cur = con.cursor()

# table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
# print(table_list)
# [('ads_by_id',), ('ads_by_query',)]

entries = [x for x in cur.execute('SELECT * FROM ads_by_query')]
# entries = [x for x in cur.execute('SELECT * FROM ads_by_id')]

# advertisers = []
# for thingo in entries:
#     advertisers.append(thingo[2])
# print(list(set(advertisers)))

# ['From the Heart', 'Fair Australia', 'The Uluru Statement from the Heart', 'Senator Jacinta Nampijinpa Price', 
# 'Not My Voice', 'Referendum News', 'Linda Burney', 'Not Enough', 'Yes23']

con.close()

In [10]:
records = []

candidate_labels = ['Advertisement about another topic', 'Advertisement about Indigenous voice to government']


counter = 0
for thingo in entries:
    counter += 1
    if counter % 50 ==0:
        
        print(counter)
    old = pd.read_csv('inter/zero_shot.csv')
    donners = old['Ad_id'].unique().tolist()
    # donners = []
    ad_id = thingo[0]

    if ad_id in donners:
        continue
    else:

        try:

            advertiser_id = thingo[1]
            name = thingo[2]
            texto = thingo[4]
            classy = classifier(texto, candidate_labels)
            
            # print(ad_id)
            # print(classy['labels'][0])
            # print(texto)
            # print(classy['scores'])
            records.append({"Ad_id": ad_id, "Advertiser": advertiser_id, "Classification": classy['labels'][0], "Text": texto})

            cat = pd.DataFrame.from_records(records)
            # dumper('inter', 'zero_shot', cat)
            new = pd.concat([old, cat])
            new.drop_duplicates(subset=['Ad_id'], inplace=True)

            dumper('inter', 'zero_shot', new)


        except Exception as e:

            print(f"Exception is {e}")
            print(f"Line: {sys.exc_info()[-1].tb_lineno}")
            pp(cat)

pp(cat)


50
Exception is object of type 'NoneType' has no len()
Line: 26
               Ad_id       Advertiser  \
0    975733956964675  104894218703531   
1    654822746614885  160347484009125   
2    639357034958709  102329728050606   
3   1349222325683428  102329728050606   
4   1012946076726810  102329728050606   
5    848264263576265  102329728050606   
6    855869672539893  102329728050606   
7    257122653770199  102329728050606   
8    607309411529234  102329728050606   
9    820465452740415  102329728050606   
10   804239254697152  102329728050606   
11  2926122184191615  102329728050606   
12  6408213595913845  102329728050606   
13   282677851017269  102329728050606   
14   808626700918542  102329728050606   
15   304380281955874  102329728050606   
16   243135325337909  102329728050606   
17   285992634116803  102329728050606   
18   849328082876755  102329728050606   
19  6452083551543155  102329728050606   
20   303206655702466  102329728050606   
21   300184352414770  102329728050

In [ ]:
zero = pd.read_csv('inter/zero_shot.csv')

zero['Count'] = 1

grp = zero.groupby(by=['Classification'])['Count'].sum().reset_index()

print(grp)

        Classification  Count
0  Voice to Parliament    432
1                other   1497
